In [65]:
import pandas as pd
import numpy as np
import torch
from sensorea_ia_lib import get_temp
from datetime import datetime
from datetime import timedelta
from IPython.display import clear_output
import time

# === Paramètres ===
file_path = ".\data\THZone2.xlsx"
CO2 = pd.read_excel(file_path,'THZone2_01')

file_conso_path = ".\data\database_room.xlsx"
database = pd.read_excel(file_conso_path,'Sheet1')

print(CO2)
print(database)


          FIFTEEN_MINUTE  internal_co2.actual
0    2025-06-15 00:00:00                433.0
1    2025-06-15 00:15:00                436.0
2    2025-06-15 00:30:00                437.0
3    2025-06-15 00:45:00                436.0
4    2025-06-15 01:00:00                441.0
...                  ...                  ...
1161 2025-06-27 02:15:00                438.0
1162 2025-06-27 02:30:00                446.0
1163 2025-06-27 02:45:00                443.0
1164 2025-06-27 03:00:00                466.0
1165 2025-06-27 03:15:00                460.0

[1166 rows x 2 columns]
               Horodatage  \
0     26/06/2025 12:09:24   
1     25/06/2025 07:25:09   
2     25/06/2025 07:15:09   
3     25/06/2025 07:05:09   
4     25/06/2025 06:55:09   
...                   ...   
2439  16/05/2025 15:08:31   
2440  15/05/2025 14:53:31   
2441  15/05/2025 14:01:05   
2442  15/05/2025 09:58:51   
2443  13/05/2025 11:16:39   

      2115_Booking_Status BACnet Trend Log - Extended Trend Log  \
0      

In [66]:
data = []
k= 0
for i in database:
    data.append(database[i])
cmd = np.transpose(data)
print(data[1])
print(np.shape(cmd))
print(cmd[0])

0       1
1       1
2       1
3       1
4       1
       ..
2439    1
2440    0
2441    1
2442    0
2443    1
Name: 2115_Booking_Status BACnet Trend Log - Extended Trend Log, Length: 2444, dtype: int64
(2444, 7)
['26/06/2025 12:09:24' 1 0.0 0.0 1.0 20.0 21.0]


### égalisation échelles de temps ###

In [67]:
normalized_cmd=[]
drop = []
for i in range(0,len(CO2['FIFTEEN_MINUTE']),1):
    # Deux dates au format chaîne de caractères
    min = timedelta(days=5, hours=0, minutes=0, seconds=0)
    for j in range(len(data[0])):
        date1 = CO2['FIFTEEN_MINUTE'][i]
        data_temp = data[0][j].replace("/","-")
        data_temp = data_temp[:6]+data_temp[8:]
        date2 = datetime.strptime(data_temp, "%d-%m-%y %H:%M:%S")
        n_min = abs(date1- date2)
        if n_min<min:
            min = n_min
            indx = j
    clear_output()
    print(CO2['FIFTEEN_MINUTE'][i])
    data_temp = data[0][indx].replace("/","-")
    data_temp = data_temp[:6]+data_temp[8:]
    print(datetime.strptime(data_temp, "%d-%m-%y %H:%M:%S"))
    print(abs(min))
    if min<timedelta(days=0, hours=0, minutes=5, seconds=0):
        normalized_cmd.append(cmd[indx])
        print("🫡 correspondance trouvée")

    else:
        drop.append(i)
        print("🫠aucune correspondance trouvée")

    print(f"😑wala j'y travaille : {float(np.shape(normalized_cmd)[0]) / float(np.shape(CO2['FIFTEEN_MINUTE'])[0]-len(drop))}")


2025-06-27 03:15:00
2025-06-26 12:09:24
0 days 15:05:36
🫠aucune correspondance trouvée
😑wala j'y travaille : 1.0


In [68]:
CO2_data = []
for i in range(len(CO2['FIFTEEN_MINUTE'])):
    if i not in drop:
        CO2_data.append([CO2['FIFTEEN_MINUTE'][i],CO2['internal_co2.actual'][i]])

In [75]:
print(len(CO2_data),len(normalized_cmd))
for i in range(len(CO2_data)):
    t = float(get_temp(CO2_data[i][0])[23])
    output = normalized_cmd[i][4]
    data_temp = normalized_cmd[i][0].replace("/","-")
    data_temp = data_temp[:6]+data_temp[8:]
    h = datetime.strptime(data_temp, "%d-%m-%y %H:%M:%S").hour
    temp = [[t],[h],normalized_cmd[i][1:4],normalized_cmd[i][5:],[float(CO2_data[i][1])]]
    input = []
    for k in temp:
        for j in k:
            input.append(j)
    data = {
            "input": input,
            "output": output
            }
    print(data)
    root = "./dataset_room/"
    name =data_temp.split(" ")[0]+data_temp.split(" ")[1].replace(":","-")
    print(name)
    path = root+name
    print(data["input"])
            # Sauvegarde dans un fichier .pt
    torch.save(data, f"{path}.pt")

826 826
{'input': [16.7, 12, 0, 0.0, 0.0, nan, 22.5, 449.0], 'output': 0.0}
15-06-2512-18-35
[16.7, 12, 0, 0.0, 0.0, nan, 22.5, 449.0]
{'input': [17.4, 13, 0, 0.0, 0.0, 24.0, 22.5, 436.0], 'output': 0.0}
16-06-2513-30-47
[17.4, 13, 0, 0.0, 0.0, 24.0, 22.5, 436.0]
{'input': [17.4, 13, 0, 0.0, 0.0, 24.0, 22.5, 437.0], 'output': 0.0}
16-06-2513-40-47
[17.4, 13, 0, 0.0, 0.0, 24.0, 22.5, 437.0]
{'input': [17.4, 14, 0, 0.0, 0.0, 24.0, 22.5, 432.0], 'output': 0.0}
16-06-2514-00-46
[17.4, 14, 0, 0.0, 0.0, 24.0, 22.5, 432.0]
{'input': [17.4, 14, 0, 0.0, 0.0, 24.0, 22.5, 431.0], 'output': 0.0}
16-06-2514-10-47
[17.4, 14, 0, 0.0, 0.0, 24.0, 22.5, 431.0]
{'input': [17.4, 14, 0, 0.0, 0.0, 24.0, 22.5, 423.0], 'output': 0.0}
16-06-2514-30-47
[17.4, 14, 0, 0.0, 0.0, 24.0, 22.5, 423.0]
{'input': [17.4, 14, 0, 0.0, 0.0, 24.5, 22.5, 430.0], 'output': 0.0}
16-06-2514-40-47
[17.4, 14, 0, 0.0, 0.0, 24.5, 22.5, 430.0]
{'input': [17.4, 15, 0, 0.0, 0.0, 24.5, 22.5, 437.0], 'output': 1.0}
16-06-2515-01-18
[17.4